## What we need 

Get data from WRDS --> is this through openAP

get data from FRED? -- Where is this? 
get fama french factors? 

How do we get the long-short positions?? like how do we see the returns?

are we able to get wrds connection or should we get a file downloaded from you 

will we be able to use gridsearchCV to run multiple models at once?

In [69]:
DEV = True # will shrink datasets after loading


In [70]:
import pandas as pd


import numpy as np

import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
import os


import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)

In [72]:
crsp = pd.read_csv('crsp_data.csv')
crsp.info()

# for speedy dev, pick 20 permnos at random (with seed)
if DEV:
    np.random.seed(123)
    permnos = crsp['permno'].unique()
    permnos = np.random.choice(permnos, size=20, replace=False)
    crsp = crsp[crsp['permno'].isin(permnos)]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4047630 entries, 0 to 4047629
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   permno  int64  
 1   date    object 
 2   ret     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 92.6+ MB


In [73]:
# crsp = pl.from_pandas(crsp)

crsp['date'] = pd.to_datetime(crsp['date'])

cleancrsp = crsp.query('ret.notnull() and date.dt.year >= 1975')
#crsp["ret"].is_not_null(), crsp['date'].dt.year() >= 1975) 
# keep only the data that has return and after 1975

len(cleancrsp['ret'].notnull()) # check that no null values in ret
len(cleancrsp['ret'].notnull()) == len(cleancrsp) # check that all ret values are not null

cleancrsp


,permno,date,ret
29180,10181,1986-05-30,6.8493
29181,10181,1986-06-30,16.6667
29182,10181,1986-07-31,1.0989
29183,10181,1986-08-29,-6.5217
29184,10181,1986-09-30,-4.6512
...,...,...,...
4037917,93235,2014-04-30,17.2319
4037918,93235,2014-05-30,-1.3066
4037919,93235,2014-06-30,5.7683
4037920,93235,2014-07-31,-2.9951


In [74]:
max_idx = cleancrsp['ret'].argmax()

cleancrsp.iloc[max_idx-5:max_idx+5] # this seems like an outlier for sure, not to be included in the data --> company is black stone minerals. 

,permno,date,ret
1789840,42076,1975-06-30,0.0000
1789841,42076,1975-07-31,8.3333
1789842,42076,1975-08-29,0.0000
1789843,42076,1975-09-30,-15.3846
1789844,42076,1975-10-31,-9.0909
2352007,61241,1975-01-31,147.0588
2352008,61241,1975-02-28,21.4286
2352009,61241,1975-03-31,33.3333
2352010,61241,1975-04-30,2.9412
2352011,61241,1975-05-30,31.4286


In [75]:
cleancrsp.sort_values('ret').tail(100) # check the top 10 returns
# these seem way too high

,permno,date,ret
2557794,69060,1983-06-30,30.4348
2352166,61241,1988-04-29,30.6818
2998916,78837,2000-03-31,30.6818
1229689,24482,2024-11-29,30.7692
2557777,69060,1982-01-29,30.7692
...,...,...,...
2557910,69060,1993-02-26,100.0000
2557783,69060,1982-07-30,100.0000
2557889,69060,1991-05-31,113.0435
1789836,42076,1975-02-28,114.2857


In [76]:
cleancrsp['ret'].describe(percentiles=[.01, .05, .10, .25, .5, .75, .9, .95, .99, .995]) # check the distribution of returns

count    2497.000000
mean        1.613311
std        15.738224
min       -66.573800
1%        -36.856788
5%        -21.481520
10%       -14.941300
25%        -6.060600
50%         0.539100
75%         7.337300
90%        17.835280
95%        26.547200
99%        53.333592
99.5%      63.974036
max       147.058800
Name: ret, dtype: float64

In [77]:
cleancrsp['permno'].nunique() # check the number of unique permnos



# profile = ProfileReport(cleancrsp, title="CRSP Data Profiling Report")


18

In [78]:
import pandas_datareader as pdr
# load Fama-French factors

start = '1975-01-01'
start_date = datetime.strptime(start, '%Y-%m-%d')

# load Fama-French 5 factors
ff_5 = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3', start=start_date)[0]
ff_mom = pdr.get_data_famafrench('F-F_Momentum_Factor', start=start_date)[0]

# # load Fama-French factors
# df_factors = pdr.ff.FamaFrenchReader('F-F_Research_Data_5_Factors_2x3', start='1975-01-01').read()[0]

# # add momentum to this

ff_mom.columns = ['Mom'] # rename
ff_mom
ff_factors = pd.merge(ff_5, ff_mom, left_index=True, right_index=True)
# df_factors # FYI: contains Mkt-RF and RF, but no Mkt
#


/var/folders/2s/rgjlyr356xq8fbht9p8_dttm0000gn/T/ipykernel_1566/839790720.py:8: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_5 = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3', start=start_date)[0]
/var/folders/2s/rgjlyr356xq8fbht9p8_dttm0000gn/T/ipykernel_1566/839790720.py:8: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_5 = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3', start=start_date)[0]
/var/folders/2s/rgjlyr356xq8fbht9p8_dttm0000gn/T/ipykernel_1566/839790720.py:9: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 

In [79]:
ff_factors

,Mkt-RF,SMB,HML,RMW,CMA,RF,Mom
Date,,,,,,,
1975-01,13.66,12.91,8.28,-0.78,-0.90,0.58,-13.82
1975-02,5.56,-0.65,-4.45,1.16,-2.11,0.43,-0.61
1975-03,2.66,4.00,2.38,1.26,-1.33,0.41,-2.04
1975-04,4.23,-0.71,-1.14,1.41,-1.34,0.44,1.38
1975-05,5.19,2.89,-4.10,-0.98,-0.60,0.44,-0.58
...,...,...,...,...,...,...,...
2024-08,1.61,-3.65,-1.13,0.85,0.86,0.48,4.79
2024-09,1.74,-1.02,-2.59,0.04,-0.26,0.40,-0.60
2024-10,-0.97,-0.88,0.89,-1.38,1.03,0.39,2.87


## Open Signals from Zip

In [80]:
from zipfile import ZipFile


def open_dataframe_from_zip(zip_filepath, csv_filename):
    """
    Opens a Pandas DataFrame from a CSV file within a zip archive.

    Args:
        zip_filepath (str): Path to the zip file.
        csv_filename (str): Name of the CSV file inside the zip archive.

    Returns:
        pandas.DataFrame: The DataFrame read from the CSV file, or None if an error occurs.
    """
    try:
        with ZipFile(zip_filepath) as zf:
            with zf.open(csv_filename) as csvf:
                df = pd.read_csv(csvf)
        return df
    except FileNotFoundError:
        print(f"Error: Zip file not found at '{zip_filepath}'")
        return None
    except KeyError:
        print(f"Error: CSV file '{csv_filename}' not found in the zip archive.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# Example usage:
if DEV == False:
    zip_filepath = 'signed_predictors_dl_wide.zip'
    csv_filename = 'signed_predictors_dl_wide.csv'
    signals = open_dataframe_from_zip(zip_filepath, csv_filename)


    
# for speedy dev,
if DEV: 
    zip_filepath = 'signals.zip'
    csv_filename = 'signals.csv'
    signals = open_dataframe_from_zip(zip_filepath, csv_filename).drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], axis=1, errors='ignore')# drop the unnamed column and date column
    # signals = signals.iloc[:,:20] # only take the first 20 columns for dev

if signals is not None:
    print(signals.tail())

         permno        date  yyyymm_signals  AM  AOP  AbnormalAccruals  \
5273595   93436  2025-02-28          202501 NaN  NaN         -0.071498   
5273596   93436  2025-03-28          202502 NaN  NaN         -0.071498   
5273597   93436  2025-04-28          202503 NaN  NaN         -0.071498   
5273598   93436  2025-05-28          202504 NaN  NaN         -0.071498   
5273599   93436  2025-06-28          202505 NaN  NaN         -0.071498   

         Accruals  AccrualsBM  Activism1  Activism2  ...  AgeIPO  \
5273595  0.043799         NaN        NaN        NaN  ...     NaN   
5273596  0.043799         NaN        NaN        NaN  ...     NaN   
5273597  0.043799         NaN        NaN        NaN  ...     NaN   
5273598  0.043799         NaN        NaN        NaN  ...     NaN   
5273599  0.043799         NaN        NaN        NaN  ...     NaN   

         AnalystRevision  AnalystValue  AnnouncementReturn  AssetGrowth  BM  \
5273595              NaN           NaN                 NaN    -0.29

In [81]:
signals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5273600 entries, 0 to 5273599
Data columns (total 21 columns):
 #   Column              Dtype  
---  ------              -----  
 0   permno              int64  
 1   date                object 
 2   yyyymm_signals      int64  
 3   AM                  float64
 4   AOP                 float64
 5   AbnormalAccruals    float64
 6   Accruals            float64
 7   AccrualsBM          float64
 8   Activism1           float64
 9   Activism2           float64
 10  AdExp               float64
 11  AgeIPO              float64
 12  AnalystRevision     float64
 13  AnalystValue        float64
 14  AnnouncementReturn  float64
 15  AssetGrowth         float64
 16  BM                  float64
 17  BMdec               float64
 18  BPEBM               float64
 19  Beta                float64
 20  BetaFP              float64
dtypes: float64(18), int64(2), object(1)
memory usage: 844.9+ MB


In [82]:


# signals.to_csv('signals.csv')

# ## want to make this a zip file to push
# with ZipFile('signals.zip', 'w') as zf:
#     zf.write('signals.csv')

In [83]:
def move_column_to_front(df, columns):
    return df[columns + [col for col in df.columns if col not in columns]]  




In [84]:

signals = signals.rename(columns={'yyyymm': 'yyyymm_signals'})


    # Create date that is one month ahead for merging with returns
signals["yyyymm_signals"] = signals["yyyymm_signals"].astype(str) 
signals["date"] = pd.to_datetime(signals["yyyymm_signals"] + "28", format="%Y%m%d") + pd.DateOffset(months=1)




In [85]:
signals = move_column_to_front(signals, ['permno', 'date']) # lag signals by a month

signals.tail()

,permno,date,yyyymm_signals,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,...,AgeIPO,AnalystRevision,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP
5273595,93436,2025-02-28,202501,NaN,NaN,-0.071498,0.043799,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.294882,NaN,NaN,NaN,NaN,NaN
5273596,93436,2025-03-28,202502,NaN,NaN,-0.071498,0.043799,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.294882,NaN,NaN,NaN,NaN,NaN
5273597,93436,2025-04-28,202503,NaN,NaN,-0.071498,0.043799,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.294882,NaN,NaN,NaN,NaN,NaN
5273598,93436,2025-05-28,202504,NaN,NaN,-0.071498,0.043799,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.294882,NaN,NaN,NaN,NaN,NaN
5273599,93436,2025-06-28,202505,NaN,NaN,-0.071498,0.043799,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.294882,NaN,NaN,NaN,NaN,NaN


In [87]:
signals.head()

,permno,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,AgeIPO,...,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP,date,yyyymm_signals
0,10000,3.681935,-161.537654,0.00008,0.031211,0.483616,14.8865,9.263089,0.075538,17.272352,...,0.751438,0.002474,-0.173642,-0.73128,2.942087,-0.037561,0.989461,0.973259,1986-02-28,198601
1,10000,3.681935,-161.537654,0.00008,0.031211,0.483616,14.8865,9.263089,0.075538,17.272352,...,0.751438,0.002474,-0.173642,-0.73128,2.942087,-0.037561,0.989461,0.973259,1986-03-28,198602
2,10000,3.681935,-161.537654,0.00008,0.031211,0.483616,14.8865,9.263089,0.075538,17.272352,...,0.751438,0.002474,-0.173642,-0.73128,2.942087,-0.037561,0.989461,0.973259,1986-04-28,198603
3,10000,3.681935,-161.537654,0.00008,0.031211,0.483616,14.8865,9.263089,0.075538,17.272352,...,0.751438,0.002474,-0.173642,-0.73128,2.942087,-0.037561,0.989461,0.973259,1986-05-28,198604
4,10000,3.681935,-161.537654,0.00008,0.031211,0.483616,14.8865,9.263089,0.075538,17.272352,...,0.751438,0.002474,-0.173642,-0.73128,2.942087,-0.037561,0.989461,0.973259,1986-06-28,198605


In [88]:
# create a new datetime column, use the year and month, but make the date 28th of the month:
cleancrsp['date'] = pd.to_datetime(cleancrsp['date'].dt.year*100 + cleancrsp['date'].dt.month, format='%Y%m') + pd.DateOffset(days=27)

/var/folders/2s/rgjlyr356xq8fbht9p8_dttm0000gn/T/ipykernel_1566/3357385141.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleancrsp['date'] = pd.to_datetime(cleancrsp['date'].dt.year*100 + cleancrsp['date'].dt.month, format='%Y%m') + pd.DateOffset(days=27)


## What we need 

Get data from WRDS --> is this through openAP

get data from FRED? -- Where is this? 
get fama french factors? 

How do we get the long-short positions?? like how do we see the returns?

are we able to get wrds connection or should we get a file downloaded from you 

will we be able to use gridsearchCV to run multiple models at once?

In [100]:
DEV = True # will shrink datasets after loading


In [101]:
import pandas as pd


import numpy as np

import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
import os


import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)

In [102]:
crsp = pd.read_csv('crsp_data.csv')
crsp.info()

# for speedy dev, pick 20 permnos at random (with seed)
if DEV:
    np.random.seed(123)
    permnos = crsp['permno'].unique()
    permnos = np.random.choice(permnos, size=20, replace=False)
    crsp = crsp[crsp['permno'].isin(permnos)]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4047630 entries, 0 to 4047629
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   permno  int64  
 1   date    object 
 2   ret     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 92.6+ MB


In [103]:
# crsp = pl.from_pandas(crsp)

crsp['date'] = pd.to_datetime(crsp['date'])

cleancrsp = crsp.query('ret.notnull() and date.dt.year >= 1975')
#crsp["ret"].is_not_null(), crsp['date'].dt.year() >= 1975) 
# keep only the data that has return and after 1975

len(cleancrsp['ret'].notnull()) # check that no null values in ret
len(cleancrsp['ret'].notnull()) == len(cleancrsp) # check that all ret values are not null

cleancrsp


,permno,date,ret
29180,10181,1986-05-30,6.8493
29181,10181,1986-06-30,16.6667
29182,10181,1986-07-31,1.0989
29183,10181,1986-08-29,-6.5217
29184,10181,1986-09-30,-4.6512
...,...,...,...
4037917,93235,2014-04-30,17.2319
4037918,93235,2014-05-30,-1.3066
4037919,93235,2014-06-30,5.7683
4037920,93235,2014-07-31,-2.9951


In [104]:
max_idx = cleancrsp['ret'].argmax()

cleancrsp.iloc[max_idx-5:max_idx+5] # this seems like an outlier for sure, not to be included in the data --> company is black stone minerals. 

,permno,date,ret
1789840,42076,1975-06-30,0.0000
1789841,42076,1975-07-31,8.3333
1789842,42076,1975-08-29,0.0000
1789843,42076,1975-09-30,-15.3846
1789844,42076,1975-10-31,-9.0909
2352007,61241,1975-01-31,147.0588
2352008,61241,1975-02-28,21.4286
2352009,61241,1975-03-31,33.3333
2352010,61241,1975-04-30,2.9412
2352011,61241,1975-05-30,31.4286


In [105]:
cleancrsp.sort_values('ret').tail(100) # check the top 10 returns
# these seem way too high

,permno,date,ret
2557794,69060,1983-06-30,30.4348
2352166,61241,1988-04-29,30.6818
2998916,78837,2000-03-31,30.6818
1229689,24482,2024-11-29,30.7692
2557777,69060,1982-01-29,30.7692
...,...,...,...
2557910,69060,1993-02-26,100.0000
2557783,69060,1982-07-30,100.0000
2557889,69060,1991-05-31,113.0435
1789836,42076,1975-02-28,114.2857


In [106]:
cleancrsp['ret'].describe(percentiles=[.01, .05, .10, .25, .5, .75, .9, .95, .99, .995]) # check the distribution of returns

count    2497.000000
mean        1.613311
std        15.738224
min       -66.573800
1%        -36.856788
5%        -21.481520
10%       -14.941300
25%        -6.060600
50%         0.539100
75%         7.337300
90%        17.835280
95%        26.547200
99%        53.333592
99.5%      63.974036
max       147.058800
Name: ret, dtype: float64

In [107]:
cleancrsp['permno'].nunique() # check the number of unique permnos



# profile = ProfileReport(cleancrsp, title="CRSP Data Profiling Report")


18

In [108]:
import pandas_datareader as pdr
# load Fama-French factors

start = '1975-01-01'
start_date = datetime.strptime(start, '%Y-%m-%d')

# load Fama-French 5 factors
ff_5 = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3', start=start_date)[0]
ff_mom = pdr.get_data_famafrench('F-F_Momentum_Factor', start=start_date)[0]

# # load Fama-French factors
# df_factors = pdr.ff.FamaFrenchReader('F-F_Research_Data_5_Factors_2x3', start='1975-01-01').read()[0]

# # add momentum to this

ff_mom.columns = ['Mom'] # rename
ff_mom
ff_factors = pd.merge(ff_5, ff_mom, left_index=True, right_index=True)
# df_factors # FYI: contains Mkt-RF and RF, but no Mkt
#


/var/folders/2s/rgjlyr356xq8fbht9p8_dttm0000gn/T/ipykernel_1566/839790720.py:8: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_5 = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3', start=start_date)[0]
/var/folders/2s/rgjlyr356xq8fbht9p8_dttm0000gn/T/ipykernel_1566/839790720.py:8: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_5 = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3', start=start_date)[0]
/var/folders/2s/rgjlyr356xq8fbht9p8_dttm0000gn/T/ipykernel_1566/839790720.py:9: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 

In [109]:
ff_factors

,Mkt-RF,SMB,HML,RMW,CMA,RF,Mom
Date,,,,,,,
1975-01,13.66,12.91,8.28,-0.78,-0.90,0.58,-13.82
1975-02,5.56,-0.65,-4.45,1.16,-2.11,0.43,-0.61
1975-03,2.66,4.00,2.38,1.26,-1.33,0.41,-2.04
1975-04,4.23,-0.71,-1.14,1.41,-1.34,0.44,1.38
1975-05,5.19,2.89,-4.10,-0.98,-0.60,0.44,-0.58
...,...,...,...,...,...,...,...
2024-08,1.61,-3.65,-1.13,0.85,0.86,0.48,4.79
2024-09,1.74,-1.02,-2.59,0.04,-0.26,0.40,-0.60
2024-10,-0.97,-0.88,0.89,-1.38,1.03,0.39,2.87


## Open Signals from Zip

In [110]:
from zipfile import ZipFile


def open_dataframe_from_zip(zip_filepath, csv_filename):
    """
    Opens a Pandas DataFrame from a CSV file within a zip archive.

    Args:
        zip_filepath (str): Path to the zip file.
        csv_filename (str): Name of the CSV file inside the zip archive.

    Returns:
        pandas.DataFrame: The DataFrame read from the CSV file, or None if an error occurs.
    """
    try:
        with ZipFile(zip_filepath) as zf:
            with zf.open(csv_filename) as csvf:
                df = pd.read_csv(csvf)
        return df
    except FileNotFoundError:
        print(f"Error: Zip file not found at '{zip_filepath}'")
        return None
    except KeyError:
        print(f"Error: CSV file '{csv_filename}' not found in the zip archive.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# Example usage:
if DEV == False:
    zip_filepath = 'signed_predictors_dl_wide.zip'
    csv_filename = 'signed_predictors_dl_wide.csv'
    signals = open_dataframe_from_zip(zip_filepath, csv_filename)


    
# for speedy dev,
if DEV: 
    zip_filepath = 'signals.zip'
    csv_filename = 'signals.csv'
    signals = open_dataframe_from_zip(zip_filepath, csv_filename).drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], axis=1, errors='ignore')# drop the unnamed column and date column
    # signals = signals.iloc[:,:20] # only take the first 20 columns for dev

if signals is not None:
    print(signals.tail())

         permno        date  yyyymm_signals  AM  AOP  AbnormalAccruals  \
5273595   93436  2025-02-28          202501 NaN  NaN         -0.071498   
5273596   93436  2025-03-28          202502 NaN  NaN         -0.071498   
5273597   93436  2025-04-28          202503 NaN  NaN         -0.071498   
5273598   93436  2025-05-28          202504 NaN  NaN         -0.071498   
5273599   93436  2025-06-28          202505 NaN  NaN         -0.071498   

         Accruals  AccrualsBM  Activism1  Activism2  ...  AgeIPO  \
5273595  0.043799         NaN        NaN        NaN  ...     NaN   
5273596  0.043799         NaN        NaN        NaN  ...     NaN   
5273597  0.043799         NaN        NaN        NaN  ...     NaN   
5273598  0.043799         NaN        NaN        NaN  ...     NaN   
5273599  0.043799         NaN        NaN        NaN  ...     NaN   

         AnalystRevision  AnalystValue  AnnouncementReturn  AssetGrowth  BM  \
5273595              NaN           NaN                 NaN    -0.29

In [111]:
signals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5273600 entries, 0 to 5273599
Data columns (total 21 columns):
 #   Column              Dtype  
---  ------              -----  
 0   permno              int64  
 1   date                object 
 2   yyyymm_signals      int64  
 3   AM                  float64
 4   AOP                 float64
 5   AbnormalAccruals    float64
 6   Accruals            float64
 7   AccrualsBM          float64
 8   Activism1           float64
 9   Activism2           float64
 10  AdExp               float64
 11  AgeIPO              float64
 12  AnalystRevision     float64
 13  AnalystValue        float64
 14  AnnouncementReturn  float64
 15  AssetGrowth         float64
 16  BM                  float64
 17  BMdec               float64
 18  BPEBM               float64
 19  Beta                float64
 20  BetaFP              float64
dtypes: float64(18), int64(2), object(1)
memory usage: 844.9+ MB


In [112]:


# signals.to_csv('signals.csv')

# ## want to make this a zip file to push
# with ZipFile('signals.zip', 'w') as zf:
#     zf.write('signals.csv')

In [113]:
def move_column_to_front(df, columns):
    return df[columns + [col for col in df.columns if col not in columns]]  




In [114]:

signals = signals.rename(columns={'yyyymm': 'yyyymm_signals'})


    # Create date that is one month ahead for merging with returns
signals["yyyymm_signals"] = signals["yyyymm_signals"].astype(str) 
signals["date"] = pd.to_datetime(signals["yyyymm_signals"] + "28", format="%Y%m%d") + pd.DateOffset(months=1)




In [115]:
signals = move_column_to_front(signals, ['permno', 'date']) # lag signals by a month

signals.tail()

,permno,date,yyyymm_signals,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,...,AgeIPO,AnalystRevision,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP
5273595,93436,2025-02-28,202501,NaN,NaN,-0.071498,0.043799,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.294882,NaN,NaN,NaN,NaN,NaN
5273596,93436,2025-03-28,202502,NaN,NaN,-0.071498,0.043799,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.294882,NaN,NaN,NaN,NaN,NaN
5273597,93436,2025-04-28,202503,NaN,NaN,-0.071498,0.043799,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.294882,NaN,NaN,NaN,NaN,NaN
5273598,93436,2025-05-28,202504,NaN,NaN,-0.071498,0.043799,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.294882,NaN,NaN,NaN,NaN,NaN
5273599,93436,2025-06-28,202505,NaN,NaN,-0.071498,0.043799,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.294882,NaN,NaN,NaN,NaN,NaN


In [116]:


# create a new datetime column, use the year and month, but make the date 28th of the month:
cleancrsp['date'] = pd.to_datetime(cleancrsp['date'].dt.year*100 + cleancrsp['date'].dt.month, format='%Y%m') + pd.DateOffset(days=27)


/var/folders/2s/rgjlyr356xq8fbht9p8_dttm0000gn/T/ipykernel_1566/2183361064.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleancrsp['date'] = pd.to_datetime(cleancrsp['date'].dt.year*100 + cleancrsp['date'].dt.month, format='%Y%m') + pd.DateOffset(days=27)


In [117]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# 1. Identify numeric columns
numeric_cols = signals.select_dtypes(include=[np.number]).columns.tolist()

# 2. Build a ColumnTransformer with mean imputer
imputer_pipeline = ColumnTransformer(
    transformers=[
        ('mean_imputer', SimpleImputer(strategy='mean'), numeric_cols)
    ],
    remainder='passthrough'  # Leave non-numeric columns (permno, date) unchanged
)

# 3. Fit and transform the signals dataframe
signals_array = imputer_pipeline.fit_transform(signals)

# 4. Rebuild the DataFrame directly into `signals`
# Carefully order columns: numeric first (imputed), then others (untouched)
all_cols = numeric_cols + [col for col in signals.columns if col not in numeric_cols]

signals = pd.DataFrame(signals_array, columns=all_cols)

# 5. Post-process: Ensure permno and date are the correct types
signals['permno'] = signals['permno'].astype(int)
signals['date'] = pd.to_datetime(signals['date'])


## Need to Convert yyyymm column to yyyy-mm-dd datetime column, rename to date

Something interesting, the crsp dataset uses trading days. How do we get this in our signals dataset. Just go off year-months?

Can we merge on yyyy-mm and not include dd so that we can match the crsp dates? But how would it know the date in the future? For instance, would it know the end of the month a year from now, or guess based on the previous crsp dd for that year or month before?

Is lagging the signals the right thing to do or do I lag the returns??


value weighting 


In [118]:
bigdata = cleancrsp.merge(signals, on=['permno', 'date'], how='left', indicator=True, validate="one_to_one") 
bigdata

,permno,date,ret,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,...,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP,yyyymm_signals,_merge
0,10181,1986-05-28,6.8493,3.681935,-161.537654,0.00008,0.031211,0.483616,14.8865,9.263089,...,0.751438,0.002474,-0.173642,-0.73128,2.942087,-0.037561,0.989461,0.973259,198604,both
1,10181,1986-06-28,16.6667,3.681935,-161.537654,0.00008,0.031211,0.483616,14.8865,9.263089,...,0.751438,0.002474,-0.173642,-0.73128,2.942087,-0.037561,0.989461,0.973259,198605,both
2,10181,1986-07-28,1.0989,3.681935,-161.537654,0.00008,0.031211,0.483616,14.8865,9.263089,...,0.751438,0.002474,-0.173642,-0.73128,2.942087,-0.037561,0.989461,0.973259,198606,both
3,10181,1986-08-28,-6.5217,3.681935,-161.537654,0.00008,0.031211,0.483616,14.8865,9.263089,...,0.751438,0.002474,-0.173642,-0.73128,2.942087,-0.037561,0.989461,0.973259,198607,both
4,10181,1986-09-28,-4.6512,3.681935,-161.537654,0.00008,0.031211,0.483616,14.8865,9.263089,...,0.751438,0.002474,-0.173642,-0.73128,2.942087,-0.037561,0.989461,0.973259,198608,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2492,93235,2014-04-28,17.2319,5.169856,-161.537654,0.00008,-0.016947,0.483616,14.8865,9.263089,...,0.751438,-0.027726,0.026716,-0.046185,0.942494,-0.000611,0.087931,0.144612,201403,both
2493,93235,2014-05-28,-1.3066,4.409937,-161.537654,0.00008,-0.016947,0.483616,14.8865,9.263089,...,0.751438,-0.027726,0.026716,-0.046185,0.942494,-0.002689,0.029387,0.193669,201404,both
2494,93235,2014-06-28,5.7683,4.468319,-161.537654,0.00008,-0.016947,0.483616,14.8865,9.263089,...,0.751438,0.008933,0.026716,-0.046185,0.942494,-0.002556,0.031067,0.210866,201405,both
2495,93235,2014-07-28,-2.9951,4.224629,-161.537654,0.00008,-0.016947,0.483616,14.8865,9.263089,...,0.751438,0.002474,0.026716,-0.046185,0.970233,-0.00308,0.042468,0.240028,201406,both


In [ ]:
signal_list = signals.columns[3:] # get the list of signals
signals[signal_list].describe()

,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,AgeIPO,AnalystRevision,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP
count,2.986693e+06,1.215192e+06,2.521764e+06,3.201801e+06,216647.000000,108731.000000,30170.000000,1.027386e+06,357438.000000,1.876115e+06,1.215192e+06,2.861710e+06,3.237201e+06,2.676641e+06,2.940113e+06,2.873411e+06,4.187358e+06,3.709235e+06
mean,3.681935e+00,-1.615377e+02,8.002906e-05,3.121143e-02,0.483616,14.886500,9.263089,7.553823e-02,17.272352,1.008518e+00,7.514384e-01,2.473509e-03,-1.736418e-01,-7.312798e-01,2.942087e+00,-3.756112e-02,9.894609e-01,9.732591e-01
std,2.688265e+01,4.798586e+04,1.551799e-01,1.406179e-01,0.499733,2.724359,12.642133,3.686100e-01,22.451399,4.864047e+00,1.553453e+00,1.013126e-01,2.696810e+00,1.041276e+00,5.282437e+01,3.385324e+02,7.430238e-01,6.358664e-01
min,0.000000e+00,-1.526718e+07,-2.815177e+00,-1.161905e+01,0.000000,6.000000,0.000000,8.399294e-07,0.000000,-1.046000e+03,-4.581968e+01,-1.608569e+00,-1.071381e+03,-1.013099e+01,-4.881022e+03,-1.210335e+05,-1.786632e+01,3.799159e-07
25%,6.427771e-01,-1.356250e+00,-5.234549e-02,-1.200336e-02,0.000000,13.000000,0.000000,4.923979e-03,5.000000,9.914712e-01,4.473222e-01,-3.780500e-02,-1.875365e-01,-1.281019e+00,3.623412e-01,-2.340956e-01,5.297898e-01,5.146013e-01
50%,1.384538e+00,-3.967994e-01,-6.826031e-03,2.897744e-02,0.000000,15.000000,7.438817,1.628784e-02,9.000000,1.000000e+00,6.884428e-01,0.000000e+00,-6.485017e-02,-5.981418e-01,6.817607e-01,-1.626706e-02,8.991139e-01,8.896496e-01
75%,3.113225e+00,-6.328331e-02,4.068833e-02,7.237209e-02,1.000000,17.000000,10.728353,5.602462e-02,18.000000,1.006897e+00,9.908170e-01,3.922600e-02,2.697890e-02,-4.733975e-02,1.168479e+00,6.726695e-02,1.330791e+00,1.309197e+00
max,1.230961e+04,2.354832e+04,7.641108e+00,2.431375e+01,1.000000,23.000000,221.282650,9.411890e+01,227.000000,5.783554e+03,3.933582e+02,9.453482e+00,1.000000e+00,4.398199e+00,1.396147e+04,4.717321e+05,5.263980e+01,1.261470e+01


In [ ]:
# pretend its a dataframe grab the rows where the column is the name and make that a list

signal_documentation = pd.read_csv('SignalDoc.csv')
signal_documentation

# need to find a way to add the column type from cat. form to match the acronyms which are the columns of the dataset
signaldoc = signal_documentation[['Acronym', 'Cat.Form']].drop_duplicates()

signaldoc.head(10)


,Acronym,Cat.Form
0,AbnormalAccruals,continuous
1,Accruals,continuous
2,AccrualsBM,discrete
3,Activism1,continuous
4,AM,continuous
5,AnalystRevision,continuous
6,AnnouncementReturn,continuous
7,AssetGrowth,continuous
8,BetaLiquidityPS,continuous
9,BetaTailRisk,continuous


In [ ]:

from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.linear_model import LogisticRegression, Ridge, Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from df_after_transform import df_after_transform
from sklearn.model_selection import KFold, cross_validate, GridSearchCV, cross_val_score


In [ ]:
## First We need to split the data into train and test sets
## Don't use train, test, split




In [ ]:
from sklearn.linear_model import Ridge


continuous_cols = signaldoc.loc[signaldoc['Cat.Form'] == 'continuous', 'Acronym'].tolist()
discrete_cols = signaldoc.loc[signaldoc['Cat.Form'] == 'discrete', 'Acronym'].tolist() # convert df to list to get columns for preprocessing pipe

numer_pipe = make_pipeline(SimpleImputer(strategy='mean'),
                           StandardScaler()) 

cat_pipe   = make_pipeline(OneHotEncoder(drop='first'))


preproc_pipe = ColumnTransformer(
    transformers=[
        ('numbers', numer_pipe, continuous_cols),  
        ('categoricals', cat_pipe, discrete_cols), 
    ],
    remainder='drop'  
)

model_pipe = make_pipeline(
    preproc_pipe,
    Ridge()
)

# model_pipe.fit(, )


In [ ]:
#Use a pipeline to preprocess the data by inputing missing values, scaling continuous features, and one-hot encoding categorical features.


,permno,date,ret,yyyymm_signals,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,...,AnalystRevision,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP,_merge
0,10181,1986-05-28,6.8493,198604,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
1,10181,1986-06-28,16.6667,198605,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
2,10181,1986-07-28,1.0989,198606,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
3,10181,1986-08-28,-6.5217,198607,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
4,10181,1986-09-28,-4.6512,198608,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both


In [ ]:
# Generate cleaned data with all of the preprocessing tools above
# preproc_pipe.fit(bigdata[for signal in signal_list if signal in bigdata], bigdata['ret'])

## get the proper columns for the model
X = bigdata[signal_list]


# OLS -- ML Style Portfolios

In [ ]:
fit_start = 1975
fit_end = 1990

## Want to find a better way to split and test the model
n_portfolios = 5



In [ ]:
signal_list

Index(['AM', 'AOP', 'AbnormalAccruals', 'Accruals', 'AccrualsBM', 'Activism1',
       'Activism2', 'AdExp', 'AgeIPO', 'AnalystRevision', 'AnalystValue',
       'AnnouncementReturn', 'AssetGrowth', 'BM', 'BMdec', 'BPEBM', 'Beta',
       'BetaFP'],
      dtype='object')

## Need to Convert yyyymm column to yyyy-mm-dd datetime column, rename to date

Something interesting, the crsp dataset uses trading days. How do we get this in our signals dataset. Just go off year-months?

Can we merge on yyyy-mm and not include dd so that we can match the crsp dates? But how would it know the date in the future? For instance, would it know the end of the month a year from now, or guess based on the previous crsp dd for that year or month before?

Is lagging the signals the right thing to do or do I lag the returns??


value weighting 


In [42]:
bigdata = cleancrsp.merge(signals, on=['permno', 'date'], how='left', indicator=True, validate="one_to_one") 
bigdata

,permno,date,ret,yyyymm_signals,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,...,AnalystRevision,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP,_merge
0,10181,1986-05-28,6.8493,198604,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
1,10181,1986-06-28,16.6667,198605,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
2,10181,1986-07-28,1.0989,198606,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
3,10181,1986-08-28,-6.5217,198607,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
4,10181,1986-09-28,-4.6512,198608,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2492,93235,2014-04-28,17.2319,201403,5.169856,NaN,NaN,-0.016947,NaN,NaN,...,NaN,NaN,-0.027726,0.026716,-0.046185,0.942494,-0.000611,0.087931,0.144612,both
2493,93235,2014-05-28,-1.3066,201404,4.409937,NaN,NaN,-0.016947,NaN,NaN,...,NaN,NaN,-0.027726,0.026716,-0.046185,0.942494,-0.002689,0.029387,0.193669,both
2494,93235,2014-06-28,5.7683,201405,4.468319,NaN,NaN,-0.016947,NaN,NaN,...,NaN,NaN,0.008933,0.026716,-0.046185,0.942494,-0.002556,0.031067,0.210866,both
2495,93235,2014-07-28,-2.9951,201406,4.224629,NaN,NaN,-0.016947,NaN,NaN,...,NaN,NaN,NaN,0.026716,-0.046185,0.970233,-0.003080,0.042468,0.240028,both


In [52]:
len(bigdata.columns)

23

In [39]:
cleancrsp.columns

Index(['permno', 'date', 'ret'], dtype='object')

In [40]:
signals.columns

Index(['permno', 'date', 'yyyymm_signals', 'AM', 'AOP', 'AbnormalAccruals',
       'Accruals', 'AccrualsBM', 'Activism1', 'Activism2', 'AdExp', 'AgeIPO',
       'AnalystRevision', 'AnalystValue', 'AnnouncementReturn', 'AssetGrowth',
       'BM', 'BMdec', 'BPEBM', 'Beta', 'BetaFP'],
      dtype='object')

In [44]:
signal_list = signals.columns[3:] # get the list of signals
signals[signal_list].describe()

,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,Activism2,AdExp,AgeIPO,AnalystRevision,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP
count,2.986693e+06,1.215192e+06,2.521764e+06,3.201801e+06,216647.000000,108731.000000,30170.000000,1.027386e+06,357438.000000,1.876115e+06,1.215192e+06,2.861710e+06,3.237201e+06,2.676641e+06,2.940113e+06,2.873411e+06,4.187358e+06,3.709235e+06
mean,3.681935e+00,-1.615377e+02,8.002906e-05,3.121143e-02,0.483616,14.886500,9.263089,7.553823e-02,17.272352,1.008518e+00,7.514384e-01,2.473509e-03,-1.736418e-01,-7.312798e-01,2.942087e+00,-3.756112e-02,9.894609e-01,9.732591e-01
std,2.688265e+01,4.798586e+04,1.551799e-01,1.406179e-01,0.499733,2.724359,12.642133,3.686100e-01,22.451399,4.864047e+00,1.553453e+00,1.013126e-01,2.696810e+00,1.041276e+00,5.282437e+01,3.385324e+02,7.430238e-01,6.358664e-01
min,0.000000e+00,-1.526718e+07,-2.815177e+00,-1.161905e+01,0.000000,6.000000,0.000000,8.399294e-07,0.000000,-1.046000e+03,-4.581968e+01,-1.608569e+00,-1.071381e+03,-1.013099e+01,-4.881022e+03,-1.210335e+05,-1.786632e+01,3.799159e-07
25%,6.427771e-01,-1.356250e+00,-5.234549e-02,-1.200336e-02,0.000000,13.000000,0.000000,4.923979e-03,5.000000,9.914712e-01,4.473222e-01,-3.780500e-02,-1.875365e-01,-1.281019e+00,3.623412e-01,-2.340956e-01,5.297898e-01,5.146013e-01
50%,1.384538e+00,-3.967994e-01,-6.826031e-03,2.897744e-02,0.000000,15.000000,7.438817,1.628784e-02,9.000000,1.000000e+00,6.884428e-01,0.000000e+00,-6.485017e-02,-5.981418e-01,6.817607e-01,-1.626706e-02,8.991139e-01,8.896496e-01
75%,3.113225e+00,-6.328331e-02,4.068833e-02,7.237209e-02,1.000000,17.000000,10.728353,5.602462e-02,18.000000,1.006897e+00,9.908170e-01,3.922600e-02,2.697890e-02,-4.733975e-02,1.168479e+00,6.726695e-02,1.330791e+00,1.309197e+00
max,1.230961e+04,2.354832e+04,7.641108e+00,2.431375e+01,1.000000,23.000000,221.282650,9.411890e+01,227.000000,5.783554e+03,3.933582e+02,9.453482e+00,1.000000e+00,4.398199e+00,1.396147e+04,4.717321e+05,5.263980e+01,1.261470e+01


In [53]:
# pretend its a dataframe grab the rows where the column is the name and make that a list

signal_documentation = pd.read_csv('SignalDoc.csv')
signal_documentation

# need to find a way to add the column type from cat. form to match the acronyms which are the columns of the dataset
signaldoc = signal_documentation[['Acronym', 'Cat.Form']].drop_duplicates()

signaldoc.head(10)


,Acronym,Cat.Form
0,AbnormalAccruals,continuous
1,Accruals,continuous
2,AccrualsBM,discrete
3,Activism1,continuous
4,AM,continuous
5,AnalystRevision,continuous
6,AnnouncementReturn,continuous
7,AssetGrowth,continuous
8,BetaLiquidityPS,continuous
9,BetaTailRisk,continuous


In [46]:

from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.linear_model import LogisticRegression, Ridge, Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from df_after_transform import df_after_transform
from sklearn.model_selection import KFold, cross_validate, GridSearchCV, cross_val_score


In [47]:
## First We need to split the data into train and test sets
## Don't use train, test, split




In [54]:
from sklearn.linear_model import Ridge


continuous_cols = signaldoc.loc[signaldoc['Cat.Form'] == 'continuous', 'Acronym'].tolist()
discrete_cols = signaldoc.loc[signaldoc['Cat.Form'] == 'discrete', 'Acronym'].tolist() # convert df to list to get columns for preprocessing pipe

numer_pipe = make_pipeline(SimpleImputer(strategy='mean'),
                           StandardScaler()) 

cat_pipe   = make_pipeline(OneHotEncoder(drop='first'))


preproc_pipe = ColumnTransformer(
    transformers=[
        ('numbers', numer_pipe, continuous_cols),  
        ('categoricals', cat_pipe, discrete_cols), 
    ],
    remainder='drop'  
)

model_pipe = make_pipeline(
    preproc_pipe,
    Ridge()
)

# model_pipe.fit(, )


In [56]:
#Use a pipeline to preprocess the data by inputing missing values, scaling continuous features, and one-hot encoding categorical features.


,permno,date,ret,yyyymm_signals,AM,AOP,AbnormalAccruals,Accruals,AccrualsBM,Activism1,...,AnalystRevision,AnalystValue,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP,_merge
0,10181,1986-05-28,6.8493,198604,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
1,10181,1986-06-28,16.6667,198605,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
2,10181,1986-07-28,1.0989,198606,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
3,10181,1986-08-28,-6.5217,198607,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both
4,10181,1986-09-28,-4.6512,198608,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both


In [49]:
# Generate cleaned data with all of the preprocessing tools above
# preproc_pipe.fit(bigdata[for signal in signal_list if signal in bigdata], bigdata['ret'])

## get the proper columns for the model
X = bigdata[signal_list]


# OLS -- ML Style Portfolios

In [50]:
fit_start = 1975
fit_end = 1990

## Want to find a better way to split and test the model
n_portfolios = 5



In [51]:
signal_list

Index(['AM', 'AOP', 'AbnormalAccruals', 'Accruals', 'AccrualsBM', 'Activism1',
       'Activism2', 'AdExp', 'AgeIPO', 'AnalystRevision', 'AnalystValue',
       'AnnouncementReturn', 'AssetGrowth', 'BM', 'BMdec', 'BPEBM', 'Beta',
       'BetaFP'],
      dtype='object')